In [1]:
import gymnasium as gym
from gymnasium import spaces
import pygame as pg
import numpy as np
from main import Game
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from datetime import datetime
import os

class MyGameEnv(gym.Env):
    def __init__(self, render_mode=True, log_mode=False, bug_mode=False, control_mode="bot", train_mode=False):
        super().__init__()
        self.render_mode = render_mode
        self.log_mode = log_mode
        self.bug_mode = bug_mode
        self.control_mode = control_mode
        self.train_mode = train_mode
        self.model = None
        self.game = None
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(
            low=np.array([0.0, 0.0, -1.0, -1.0, 0.0], dtype=np.float32),
            high=np.array([1.0, 1.0, 1.0, 1.0, 1.0], dtype=np.float32),
            dtype=np.float32
        )
    def game_setting(self):
        self.game = Game(model= self.model, render_mode = self.render_mode, log_mode=self.log_mode, bug_mode=self.bug_mode, control_mode=self.control_mode, train_mode=self.train_mode)

    def reset(self, seed=None, options=None):
        obs = self.game.reset()
        return np.array(obs, dtype=np.float32), {}

    def step(self, action):
        obs, reward, done, info = self.game.step(action)

        if done: # step마다 reward 기록용
            print(f"[EP DONE] obs: {obs}, reward: {reward}")
        return np.array(obs, dtype=np.float32), reward, done, False, info

    def render(self):
        if self.render_mode:
            self.game.draw()

    def close(self):
        self.game.quitgame()

    def train(self):
        self.render_mode = False
        self.log_mode = False
        self.bug_mode = False
        self.control_mode = "bot"
        self.train_mode = True
        self.game_setting()

        # check_env(env, warn=True)  # 환경이 gym 스타일로 잘 되어 있는지 체크
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_name = f"dqn_model_{timestamp}"
        model_name = os.path.join("./bot", model_name)

        model = DQN("MlpPolicy", self, verbose=1, tensorboard_log="./dqn_logs")
        model.learn(total_timesteps=50000, tb_log_name="run_1")
        os.makedirs("./bot", exist_ok=True)

        model.save(model_name)
        self.model = model
        # self.close()

    def check_env(self):
        check_env(self)
        # self.close()
    def load_model(self, model_name):
        self.model = DQN.load(os.path.join("./bot", model_name))
        
    def play_game_with_bot(self):
        assert self.control_mode == "bot"
        assert self.model is not None
        assert self.train_mode == False
        self.game_setting()
        self.game.show_start_screen()
        while self.game.running:
            self.game.new()
            self.game.show_go_screen()
        self.game.quitgame()


In [3]:
env.close()

In [ ]:
#env 설정 제대로 되었는지 확인.
env = MyGameEnv()
check_env(env)
env.close()

In [ ]:
#모델 훈련
env = MyGameEnv()
env.train()
env.close()

공이 안움직이는 것 같다... 문제를 해결해보자. 05/22/12:06. //
step이 길어질수록 reward를 작게 하자. 벽에 걸려서 아예 움직이지 못하는 이슈가 있네요.

05/25/16:41 봇은 대충 만들었어요. show_go_screen 화면이 나오지 않는 현상을 해결해야 합니다.

In [2]:
#모델 로드 및 실제 플레이 화면 도출.
env = MyGameEnv(render_mode=True, log_mode=True, bug_mode=True, control_mode="bot", train_mode=False)
env.load_model(model_name = "dqn_model_20250525_140415.zip")
env.play_game_with_bot()

error: display Surface quit